In [9]:
!pip install youtube_transcript_api
!pip install openai
!pip install gradio

In [10]:
import re
import json
import openai
import gradio as gr
from youtube_transcript_api import YouTubeTranscriptApi

In [11]:
class Summarizer:
    def __init__(self, link):
        self.link = link
        self.count = 0

    def youtube_parser(self):
        reg_exp = r'^.*((youtu.be/)|(v/)|(u/\w/)|(embed/)|(watch\?))\??v?=?([^#&?]*).*'
        match = re.match(reg_exp, self.link)
        return match.group(7) if match and len(match.group(7)) == 11 else False

    def get_transcript(self):
        video_id = self.youtube_parser()
        if video_id:
            try:
                ts = YouTubeTranscriptApi.get_transcript(video_id)
                transcript = " ".join([entry['text'] for entry in ts])
                return transcript
            except Exception as e:
                return f"An error occurred: {e}"
        else:
            return "Please provide a correct YouTube URL."

    def chat_with_model(self, model="gpt-3.5-turbo", user_question=""):
        transcript = self.get_transcript()

        with open('/content/API_key.json') as f:
            data = json.load(f)

        openai.api_key = data['openai_api_key']

        messages = [{"role": "system", "content": "You are the helpful and kind assistant that answers the user's doubt"},
                    {"role": "user", "content": user_question},
                    {"role": "assistant", "content": f"Answer the user's questions from this transcript, if the user asks anything that is not related to the transcript then answer that you do no know that:\n{transcript}"}]

        response = openai.ChatCompletion.create(
            model=model,
            messages=messages,
            temperature=0.3,
        )

        return response.choices[0].message["content"]

def summarizer(link, user_question=""):
    summ = Summarizer(link)
    response = summ.chat_with_model(user_question=user_question)
    return response

# Example usage
# video_link = 'https://www.youtube.com/watch?v=3kVWye29DdU'
# user_question = "Can you explain the main points of the video?"


In [14]:
count = 0
while True:
  if count == 0:
    link_inp = input("Enter the link of the video: ")
    user_inp = input("\nEnter the questions you want to ask: ")
    count = 1
  else:
    user_inp = input("\nEnter the questions you want to ask: ")
    if user_inp == 'exit':
      break
  summar = summarizer(link_inp, user_question = user_inp)
  print(summar)

Enter the link of the video: https://www.youtube.com/watch?v=Ea1_1aVfwl4

Enter the questions you want to ask: What is this video about?
The video is about explaining the concept of exponentiation and the difference between writing an exponent on the right-hand side and the left-hand side. The video uses the example of titration with the number 10 to illustrate the concept. It shows how increasing the exponent can result in significantly larger numbers and discusses the implications of extremely large numbers. The video also challenges the viewer to figure out the value of a specific number without providing the answer.

Enter the questions you want to ask: key points of this video
The key points of this video are:

1. The video discusses the concept of titration, which involves writing numbers using exponents.
2. The exponent on the right-hand side indicates repeated multiplication. For example, 10^2 means multiplying 10 by itself twice, resulting in 100.
3. Increasing the exponent si

In [ ]:
summarizer

'This video is about studying smarter, not harder. It provides tips and strategies for students to improve their study habits and become more efficient and effective in their learning. The video covers topics such as taking breaks during study sessions, creating a designated study area, understanding concepts rather than just memorizing facts, and the importance of getting enough sleep for optimal brain function. The video also mentions that there will be a part two with four more study tips.'

In [ ]:
# import gradio as gr

# def summarizer(link, user_question):
#     summ = Summarizer(link)
#     response = summ.chat_with_model(user_question=user_question)
#     return response

# iface = gr.Interface(
#     fn=summarizer,
#     inputs=[
#         gr.inputs.Textbox(label="YouTube Video URL"),
#         gr.inputs.Textbox(label="Your Question")
#     ],
#     outputs=gr.outputs.Textbox(label="Summary"),
#     live=True,
#     layout="vertical"
# )

# iface.launch(share=False)


<ipython-input-7-137a6d499c4a>:11: GradioDeprecationWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  gr.inputs.Textbox(label="YouTube Video URL"),
<ipython-input-7-137a6d499c4a>:11: GradioDeprecationWarning: `optional` parameter is deprecated, and it has no effect
  gr.inputs.Textbox(label="YouTube Video URL"),
<ipython-input-7-137a6d499c4a>:11: GradioDeprecationWarning: `numeric` parameter is deprecated, and it has no effect
  gr.inputs.Textbox(label="YouTube Video URL"),
<ipython-input-7-137a6d499c4a>:12: GradioDeprecationWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  gr.inputs.Textbox(label="Your Question")
<ipython-input-7-137a6d499c4a>:12: GradioDeprecationWarning: `optional` parameter is deprecated, and it has no effect
  gr.inputs.Textbox(label="Your Question")
<ipython-input-7-137a6d499c4a>:12

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>